### This Code is for Backend Developer
- Extract only code that uses tensorflow lite from ‘./resnet-all-datasets.ipynb’
- Modified the code so that images downloaded from an external URL can be used.
- The final code ported to the backend can be found at this [link](https://github.com/BloomMate/BloomMate-BE/blob/main/plants/utils.py#L45)

In [2]:
import tensorflow as tf
import numpy as np

height,width=180,180

In [3]:
class_names = [ 'Corn___Common_rust', 
                'Corn___Gray_leaf_spot', 
                'Corn___Northern_Leaf_Blight', 
                'Corn___healthy', 
                'Potato___Early_blight', 
                'Potato___Late_blight', 
                'Potato___healthy', 
                'Strawberry___Leaf_scorch', 
                'Strawberry___healthy', 
                'Tomato___Early_blight', 
                'Tomato___Late_blight',
                'Tomato___Target_Spot', 
                'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 
                 'Tomato___healthy']

In [29]:
import os
from keras.preprocessing import image
from PIL import Image
# 로컬 이미지 경로 설정
url = "https://res.cloudinary.com/dolc0qkxk/image/upload/v1699615628/KakaoTalk_20231110_202630870_sr98tx.png"

os.system("curl " + url + " > test2.jpg")

# 이미지 불러오기
local_image_path = "./test2.jpg"

# 이미지 불러오기
img = image.load_img(local_image_path, target_size=(height, width))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # 배치 차원 추가

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  245k  100  245k    0     0  1141k      0 --:--:-- --:--:-- --:--:-- 1168k


In [30]:
PATH_TO_MODEL = './tensorflow-lite-models/model-231025 (resnet).tflite'

interpreter = tf.lite.Interpreter(model_path=PATH_TO_MODEL)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data.
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

[[4.3900849e-07 6.1759566e-09 2.6285302e-10 9.8482235e-07 2.4985874e-10
  3.0427866e-10 1.1133936e-09 4.4969410e-09 2.0610912e-07 6.3493439e-06
  9.9999154e-01 1.8944957e-07 1.4728759e-07 9.2554188e-08]]


In [31]:
interpreter.get_signature_list()

{'serving_default': {'inputs': ['resnet50_input'], 'outputs': ['dense_1']}}

In [32]:
classify_lite = interpreter.get_signature_runner('serving_default')
classify_lite

predictions_lite = classify_lite(resnet50_input = img_array)['dense_1']
score_lite = tf.nn.softmax(predictions_lite)

# 클래스 이름과 신뢰도 출력
print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(predictions_lite)], 100 * np.max(predictions_lite))
)

This image most likely belongs to Strawberry___healthy with a 99.30 percent confidence.
